In [1]:
import os
from pathlib import Path

root_directory = "/cluster/home/wueestm/f3loc/"
os.chdir(root_directory)
root_directory
Path.cwd()

PosixPath('/cluster/home/wueestm/f3loc')

In [6]:
import os
import cv2
import numpy as np
from torch.utils.data import Dataset
from utils.utils import gravity_align, gravity_align_depth, gravity_align_normal, gravity_align_segmentation
from metric3d.hubconf import *
from src.depth_image_functions import *
import yaml
from attrdict import AttrDict

In [7]:
dataset = "hge_customized_complete" #"hge_customized_cropped"
dataset_path = "/cluster/project/cvg/data/lamar/HGE_customized_complete" #"/cluster/project/cvg/data/lamar/HGE_customized_cropped"
dataset_dir = os.path.join(dataset_path, dataset)
split_file = os.path.join(dataset_dir, "split.yaml")
with open(split_file, "r") as f:
    split = AttrDict(yaml.safe_load(f))

In [8]:
# train_dataset = GridSeqDataset_hge_customized_metric3d(
#     dataset_dir,
#     split.train,
#     L=L,
#     depth_dir=depth_dir,
#     depth_suffix=depth_suffix,
#     add_rp=add_rp,
#     roll=roll,
#     pitch=pitch,
# )

In [24]:
scene_names = split.train
start_scene = None
end_scene = None
L = 0
scene_start_idx = []
gt_depth = []
gt_pose = []
rgbs = []
K = np.array([[1596, 0, 960], [0, 1596, 720], [0, 0, 1]])

In [25]:
scene_start_idx.append(0)
start_idx = 0
print("scene_names: ", scene_names)
for scene in scene_names:
    print("scene: ", scene)

    # read pose
    pose_file = os.path.join(dataset_dir, scene, "poses.txt")
    with open(pose_file, "r") as f:
        poses_txt = [line.strip() for line in f.readlines()]
    traj_len = len(poses_txt)

    start_idx += traj_len // (L + 1)
    scene_start_idx.append(start_idx)


scene_names:  ('ios_2022-01-12_15.15.53_000', 'ios_2022-01-18_17.05.03_000', 'ios_2022-01-20_16.52.33_001', 'ios_2022-01-25_14.34.24_002', 'ios_2022-01-25_14.57.49_000', 'ios_2022-01-25_15.13.54_000', 'ios_2022-06-13_10.45.07_000', 'ios_2022-06-13_15.59.36_000', 'ios_2022-06-14_17.12.28_000', 'ios_2022-06-30_15.55.53_000', 'ios_2022-07-01_15.18.09_000', 'ios_2022-07-01_15.58.10_000')
scene:  ios_2022-01-12_15.15.53_000
scene:  ios_2022-01-18_17.05.03_000
scene:  ios_2022-01-20_16.52.33_001
scene:  ios_2022-01-25_14.34.24_002
scene:  ios_2022-01-25_14.57.49_000
scene:  ios_2022-01-25_15.13.54_000
scene:  ios_2022-06-13_10.45.07_000
scene:  ios_2022-06-13_15.59.36_000
scene:  ios_2022-06-14_17.12.28_000
scene:  ios_2022-06-30_15.55.53_000
scene:  ios_2022-07-01_15.18.09_000
scene:  ios_2022-07-01_15.58.10_000


In [28]:
if start_scene is None:
    N = scene_start_idx[-1]
else:
    # compute N
    N = (
        scene_start_idx[end_scene + 1]
        - scene_start_idx[start_scene]
    )
print(scene_start_idx)

[0, 579, 733, 1083, 1163, 1647, 1869, 2359, 2700, 2963, 3308, 3615, 3820]


In [30]:
N

3820

In [32]:
start_scene

In [ ]:
for idx in range(N):
    if start_scene is not None:
        idx += scene_start_idx[start_scene]

    # get the scene name according to the idx
    scene_idx = np.sum(idx >= np.array(scene_start_idx)) - 1
    scene_name = scene_names[scene_idx]

    # get idx within scene
    idx_within_scene = idx - scene_start_idx[scene_idx]

    # get reference image
    
    image_path = os.path.join(
        dataset_dir,
        scene_name,
        "rgb",
        str(idx_within_scene).zfill(5) + "-0" + ".jpg",
    )
    ref_img = cv2.imread(image_path, cv2.IMREAD_COLOR).astype(
        np.float32
    )  # (H, W, 3)

    # get depth image
    depth_img = get_depth_image_metric3d(ref_img)
    depth_img = depth_img.cpu().numpy()  # Move to CPU and convert to numpy array
    #print("Depth image done.")
